In [58]:
import numpy as np
import pandas as pd
import sys
from joblib import Parallel, delayed
from scipy.stats import mode
from importlib import import_module
import os.path
from scipy.stats import mode

sys.path.append('.')

## Peaking the APS solution - Example Figure 3

In [51]:
p = import_module(f'data.prob1')

In [65]:
def propose(x_given, x_values):
    if x_given == x_values[0]:
        return( np.random.choice([x_values[1], x_values[-1]],
        p=[0.5, 0.5]) )

    if x_given == x_values[-1]:
        return( np.random.choice([x_values[0], x_values[-2]],
        p=[0.5, 0.5]) )

    idx = list(x_values).index(x_given)
    return( np.random.choice([x_values[idx+1], x_values[idx-1]],
    p=[0.5, 0.5]) )

def propose2(a_sim, a_values):
    return np.random.choice(a_values)


In [93]:
def iter_mcmc_defender(J, propose, d_sim, theta_sim, a_star):
    
    d_tilde = propose(d_sim, p.d_values)
    a_tilde = np.ones(J, dtype='int') * a_star[p.d_values == d_tilde]
    theta_tilde = np.array([p.prob(d_tilde, a) for a in a_tilde])[:,0]

    num = p.d_util(d_tilde, theta_tilde)

    den = p.d_util(d_sim, theta_sim)

    quot = np.prod(num/den)

    if np.random.uniform() <= quot:
        return d_tilde, theta_tilde
    else:
        return d_sim, theta_sim
    
def solve_defender(J_max, a_star, propose, burnin):
    J = 1
    J_grid = range(1, J_max)
    d_sim = np.zeros(len(J_grid), dtype = int)
    d_sim[0] = np.random.choice(p.d_values)
    a_sim = np.ones(J, dtype='int') * a_star[p.d_values == d_sim[0]]
    theta_sim = np.array([p.prob(d_sim[0], a) for a in a_sim])[:,0]
    ##
    for i, J in enumerate(J_grid):

        d_sim[i], theta_sim = iter_mcmc_defender(J, propose, d_sim[i-1], theta_sim, a_star)
        theta_sim = np.append(theta_sim, np.random.choice(theta_sim))
        if J%10 == 0:
            burnin = int(0.2*i)
            print( mode(d_sim[burnin:i])[0][0] ) 
            dist = pd.Series(d_sim[burnin:i])
            name = 'results/prob1_EC/prob1_adg_peaked' + str(J) + '.csv'
            dist.to_csv(name, index = False)

In [95]:
a_star = np.array([1,1,1,1,1,1,1,1,0,0])
solve_defender(500, a_star, propose2, burnin=0.2)

6
6
8
8
8
8
9


/home/roi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


9
9
9
9
9
9
9
9
9
9
9
8
8
9
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


## Peaking the APS solution - Example Figure 7

In [110]:
def iter_mcmc_defender(J, propose, d_sim, theta_sim, p_ad):
    d_tilde = propose(d_sim, p.d_values)
    a_tilde = np.random.choice(p.a_values, p=p_ad[p.d_values == d_tilde, :][0], size = J)
    theta_tilde = np.array([p.prob(d_tilde, a) for a in a_tilde])[:,0]

    num = p.d_util(d_tilde, theta_tilde)

    den = p.d_util(d_sim, theta_sim)

    quot = np.prod(num/den)

    if np.random.uniform() <= quot:
        return d_tilde, theta_tilde
    else:
        return d_sim, theta_sim

    
# With Muller's trick
def solve_defender(J_max, p_ad, propose, burnin):
    J = 1
    J_grid = range(1, J_max)
    d_sim = np.zeros(len(J_grid), dtype = int)
    d_sim[0] = np.random.choice(p.d_values)
    a_sim = np.random.choice(p.a_values, p=p_ad[p.d_values == d_sim[0], :][0], size = J)
    theta_sim = np.array([p.prob(d_sim[0], a) for a in a_sim])[:,0]
    ##
    for i, J in enumerate(J_grid):

        d_sim[i], theta_sim = iter_mcmc_defender(J, propose, d_sim[i-1], theta_sim, p_ad)
        theta_sim = np.append(theta_sim, np.random.choice(theta_sim))
        if J%10 == 0:
            burnin = int(0.2*i)
            print( mode(d_sim[burnin:i])[0][0] ) 
            dist = pd.Series(d_sim[burnin:i])
            name = 'results/prob1_EC/prob1_ara_peaked' + str(J) + '.csv'
            dist.to_csv(name, index = False)

In [111]:
p_ad = pd.read_csv('results/1586874387_prob1_aps_ara_pa.csv', index_col=['d'])
solve_defender(500, p_ad.values, propose2, burnin=0.2)

9
9
8
8
8
9
9
9


/home/roi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
